# Your First RAG Application: Stone Ridge Investor Letter Assistant

> **Note:** Please follow the best practices outlined in the [SRHG AI Usage Guidelines](https://srhg.enterprise.slack.com/docs/T0HANKTEC/F0AB86J3A1L).

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Stone Ridge Investor Letter Assistant**.

Imagine having an AI assistant that can answer your questions about Stone Ridge's investment philosophy, market insights, and strategic outlook based on their investor letters - that's exactly what we'll build here. We'll be using local embeddings via HuggingFace's sentence-transformers and Anthropic's Claude for the LLM.

> NOTE: This was done with Python 3.12.3.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading the Stone Ridge Investor Letter)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Investor Letter Assistant)
  - 🚧 Activity #1: Enhance Your Investor Letter Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
import os
import certifi

# Create a combined cert bundle with Zscaler for corporate network
zscaler_cert = "/Users/ari.packer/repos/sidekick/zscaler.pem"
combined_cert = "/tmp/combined_certs.pem"

with open(combined_cert, "w") as outfile:
    with open(certifi.where(), "r") as certifi_file:
        outfile.write(certifi_file.read())
    with open(zscaler_cert, "r") as zscaler_file:
        outfile.write(zscaler_file.read())

os.environ['REQUESTS_CA_BUNDLE'] = combined_cert
os.environ['SSL_CERT_FILE'] = combined_cert
os.environ['CURL_CA_BUNDLE'] = combined_cert

from aimakerspace.text_utils import PDFFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our PDF file into a single document in memory.

Let's look at the relevant bits of the `PDFFileLoader` class:

```python
def load_file(self):
        doc = pymupdf.open(self.path)
        text = ""
        for page in doc:
            text += page.get_text()
        self.documents.append(text)
```

We're loading the PDF document using PyMuPDF, extracting text from each page, and storing that output in our `self.documents` list.

> NOTE: We're using the Stone Ridge 2025 Investor Letter as our sample data. This content covers investment philosophy, market analysis, and strategic insights - perfect for building an investor letter assistant!

In [3]:
pdf_loader = PDFFileLoader("data")
documents = pdf_loader.load_documents()
len(documents)

3

In [4]:
# Preprocess the document to remove unwanted sections
# Risk disclosures were causing irrelevant context retrievals
documents[0] = documents[0].split('Risk Disclosures')[0]
documents[1] = documents[1].split('Risk Disclosures')[0]
documents[2] = documents[2].split('Risk Disclosures')[0]


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter(1000, 0, 'paragraph')
split_documents = text_splitter.split_texts(documents)
print(len(split_documents))

181


Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[50:56]

['Stone Ridge is most proud of the 50/50 partnership we have with you, our investors. We are on the path together. \n\nYou contribute the capital necessary to propel and sustain groundbreaking product development. We contribute \n\nour collective careers’ worth of experience in sourcing, structuring, execution, and risk management. Together, it \n\nworks. In that spirit, I offer my deepest gratitude to you for sharing responsibility for your wealth with us this year. \n\nWe look forward to serving you again in 2024.\n\nWarmly,\n\nRoss L. Stevens\n\nFounder, CEO\n\nEndnotes\n\ni\t 2023 returns as of 12/29/2023.\n\nii\t Source: Credit Suisse, Deloitte, Masterworks, McKinsey.\n\niii\t Source: Art Basel UBS 2022. Volume is auction volume plus dealer volume. Auction volumes are actuals. Dealer volumes are \n\nestimated by scaling auction volume by the market-wide proportion of auction versus dealer volume.',
 'iv\t Source for art returns: Index of public auction sales of PWC art constructed

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### Embedding Model Setup

Our embeddings are generated locally using the `all-MiniLM-L6-v2` model from HuggingFace's sentence-transformers library. No API key is required for embeddings!

In [7]:
# Embeddings run locally - no API key needed for this step!

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which uses the `all-MiniLM-L6-v2` model from HuggingFace's sentence-transformers library, running locally on your machine.

> **Quick Info About `all-MiniLM-L6-v2`**:
> - It is a lightweight, fast embedding model
> - It returns vectors with dimension **384**
> - No API calls required - runs entirely locally

#### ❓Question #1:

The default embedding dimension of `all-MiniLM-L6-v2` is 384, as noted above. 

1. What are some trade-offs between smaller embedding dimensions (like 384) vs larger ones (like 1536)?
2. How might you choose an embedding model for a specific use case?

> NOTE: Consider factors like speed, memory usage, and semantic accuracy when thinking about these questions!

##### ✅ Answer:
1. Smaller embedding dimensions use less resources (e.g., memory, compute) for the process. Smaller models behind APIs like OpenAI's are probably less expensive to use and faster. Larger models are more expensive (resource-use local, cost remote), but return more accurate results.
2. I would go with the smallest model that is semantically accurate enough for the use case to save resources. Determining what is semantically accurate enough is difficult, and would probably require controlled testing (e.g., write a program that does a blind test by testing the same N queries against multiple models and having the final result be rated). This would allow determining if the incremental cost of the larger models provides actual value to the end result to help with a cost/benefit analysis.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, self.get_embeddings, list_of_text)
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
The benefits are obvious when the embedding model is behind a remote server, because each request that leaves the process and hits the network adds significant latency on top of the time it takes the model to actually compute the result. Embedding each chunk sequentially in this manner would be very slow, and performing remote calls asynchronously and concurrently is a basic technique used by software systems to increase throughput.

The model we're using here is not on a remote server, so the main reason to perform embedding concurrently would be to allow multiple threads to do work across multiple cores on the local machine. The HuggingFace embedding model already effectively uses multithreading, so there is little real benefit in this instance from the async approach.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We embed each document using the local `all-MiniLM-L6-v2` model
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is Stone Ridge's investment philosophy?", k=3)

[('for our investors across all franchises, with no down months. While everyone at Stone Ridge can be proud of those \n\nnumbers, perspective is vital. \n\nFirst, nothing we’ve achieved this year is because of this year. Success, and failure, are lagging indicators. Mountain \n\nclimbing disasters are always a series of small, seemingly inconsequential decisions that interact in unexpected \n\nways, compounding exponentially. Success works the same way. \n\nIt is fashionable these days to say investment outcomes follow a Power Law. That’s technically true and soulless. At \n\nStone Ridge, the compounding we’re after is in wisdom and trust in our relationships with each other. Life is much \n\nmore satisfying when we realize that relationships also follow a Power Law, and invest accordingly.\n\nSecond, we let the simplicity of our business model be enough. The humility of our questions and the plainness of our',
  np.float64(0.6605797911864041)),
 ('particularly around known and unknown

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how chat models work with the Messages API format.

Most LLM APIs (OpenAI, Anthropic, etc.) use a similar role-based message structure:

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation.
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### Chat Model

Next we have our chat model wrapper, which provides a consistent interface for interacting with Anthropic's Claude.

Let's take a peek at how that is constructed:

```python
class ChatAnthropic:
    def __init__(self, model_name: str = "claude-sonnet-4-5-20250929"):
        self.model_name = model_name
        self.base_url = os.getenv("ANTHROPIC_BASE_URL")
        self.auth_token = os.getenv("ANTHROPIC_AUTH_TOKEN")
        if self.auth_token is None:
            raise ValueError("ANTHROPIC_AUTH_TOKEN is not set")

    def run(self, messages, text_only: bool = True, **kwargs):
        # ... handles system messages separately (Anthropic API requirement)
        # ... creates Anthropic client and returns response
```

> NOTE: Anthropic's API handles system messages differently than OpenAI - they're passed as a separate parameter rather than in the messages list.

#### ❓ Question #3:

When calling an LLM API - are there any ways we can achieve more reproducible outputs?

> NOTE: Look into parameters like `temperature` and `seed` that most LLM providers support!

##### ✅ Answer:
I wasn't familiar with using temperature to control the randomness of output. To achieve more reproducible outputs, temperature can be set to 0 to select the most probable token at each step (but variations still occur for reasons I don't fully understand). In order for this to work, a seed must be provided and kept consistent across each request.

### Creating and Prompting the LLM

Let's tie all these together and use it to prompt our chat model!

In [10]:
from aimakerspace.ai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.ai_utils.chatmodel import ChatAnthropic

chat_llm = ChatAnthropic()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_llm.run(messages)

{'model': 'claude-sonnet-4-5-20250929', 'max_tokens': 1024, 'messages': [{'role': 'user', 'content': 'What is the best way to write a loop?'}], 'system': 'You are an expert in Python, you always answer in a kind way.'}


In [11]:
print(response)

# Best Ways to Write Loops in Python 🐍

The "best" way depends on your specific use case! Here are the most Pythonic approaches:

## **1. For Loops with Iterables (Most Common)**
```python
# Simple iteration
for item in my_list:
    print(item)

# With index using enumerate()
for index, item in enumerate(my_list):
    print(f"{index}: {item}")
```

## **2. List Comprehensions (When Building Lists)**
```python
# Instead of:
result = []
for x in range(10):
    result.append(x * 2)

# Use:
result = [x * 2 for x in range(10)]
```

## **3. While Loops (For Conditional Iteration)**
```python
while condition:
    # do something
    pass
```

## **4. Built-in Functions (Often the Best Choice!)**
```python
# Instead of loops, consider:
sum(my_list)           # Sum elements
any(my_list)           # Check if any True
all(my_list)           # Check if all True
map(func, my_list)     # Apply function to all
filter(func, my_list)  # Filter elements
```

## **General Tips:**
- ✅ Use `for` loops for k

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [12]:
RAG_SYSTEM_TEMPLATE = """You are a helpful investor letter assistant that answers questions about Stone Ridge's investment philosophy, market insights, and strategic outlook based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in the investor letter"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a reminder that this is for informational purposes only and not investment advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [13]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatAnthropic, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 3, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)        

        context_prompt = ""
        similarity_scores = []
        
        # sort context list by score descending
        context_list = sorted(context_list, key=lambda x: x[1], reverse=True)

        for i, (context, score) in enumerate(context_list, 1):

            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [14]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_llm,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is Stone Ridge's investment philosophy?",
    k=3,
    response_length="efficient", 
    include_warnings=True,
    confidence_required=True
)

print(result['prompts_used']["user"])

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

{'model': 'claude-sonnet-4-5-20250929', 'max_tokens': 1024, 'messages': [{'role': 'user', 'content': "Context Information:\n[Source 1]: for our investors across all franchises, with no down months. While everyone at Stone Ridge can be proud of those \n\nnumbers, perspective is vital. \n\nFirst, nothing we’ve achieved this year is because of this year. Success, and failure, are lagging indicators. Mountain \n\nclimbing disasters are always a series of small, seemingly inconsequential decisions that interact in unexpected \n\nways, compounding exponentially. Success works the same way. \n\nIt is fashionable these days to say investment outcomes follow a Power Law. That’s technically true and soulless. At \n\nStone Ridge, the compounding we’re after is in wisdom and trust in our relationships with each other. Life is much \n\nmore satisfying when we realize that relationships also follow a Power Law, and invest accordingly.\n\nSecond, we let the simplicity of our business model be enough.

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
To make the LLM have a more thoughtful, detailed response, we can use Chain-of-Thought Prompting, as described in the assigned reading "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models." This builds on few-shot prompting to provide examples that effectively have the LLM produce a result that reasons instead of just coming up with an answer. For example, if I say "what's 2+4-1?", the LLM might say "7", or it might not (probably will since this is a simplified example). But if I give an example of how to solve it in context that says "This equation can be modified to ((2+4)-1) and executed in steps. (2+4) is (6). (6-1) is 5. The answer is 5." then the model will follow that line of reasoning on other problems and be more likely to get the answer right.

### 🏗️ Activity #1:

Enhance your Stone Ridge Investor Letter Assistant in some way! 

Suggestions are: 

- **Multi-Document Support**: Allow it to ingest multiple investor letters from different years for historical comparison
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for financial content?
- **Metadata Support**: Add metadata like year, topic categories (market outlook, strategy, performance) to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for financial content?
- **Multi-Source Ingestion**: Add the capability to ingest content from SEC filings, earnings calls, or other financial documents

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this investor letter assistant most useful for understanding Stone Ridge's investment approach.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:
### YOUR CODE HERE

# =============================================================================
# PREAMBLE
# =============================================================================
#
# This solution implements several enhancements to the base RAG application:
#
# 1. PARAGRAPH-BASED CHUNKING (EXPERIMENTAL - DID NOT WORK WELL)
#    Added a new "paragraph" split mode to CharacterTextSplitter. Since PDF
#    extraction doesn't reliably preserve paragraph boundaries, we use " \n"
#    (space followed by newline) as an approximation. However, it was not
#    possible to accurately split by paragraphs with the output of the PDF
#    parser. Single newlines within chunks are replaced with literal \n.
#
# 2. DATA PREPROCESSING/CLEANUP
#    Pre-filtered documents to remove "Risk Disclosures" sections before chunking.
#    Initial results were poor because legal boilerplate dominated the retrieved
#    context. Other approaches tried (exclusion searches, downranking legalese
#    keywords) were less effective than simply removing disclaimers upfront.
#
# 3. MULTI-DOCUMENT SUPPORT
#    Added additional investor letters (2023, 2024, 2025) to the data directory
#    and pointed PDFFileLoader to the parent directory. This enables historical
#    comparison across multiple years without modifying the loader itself.
#
# 4. ALTERNATIVE DISTANCE METRIC
#    Added toggle between cosine similarity and Pearson correlation via the
#    DISTANCE_METRIC constant. Both performed similarly on this dataset - cosine
#    is generally optimal for normalized embeddings, but Pearson can capture
#    relationships cosine might miss.
#
# 5. SOURCE METADATA
#    Added metadata support to track which document each chunk originated from.
#    The VectorDatabase now stores (vector, metadata) tuples, and search results
#    include source information. Context sent to the LLM now shows [Source: filename]
#    for each chunk, enabling the model to cite specific documents in its answers.
#
# 6. PER-DOCUMENT CONTEXT RETRIEVAL
#    Added CONTEXT_PER_DOCUMENT flag to retrieve N chunks from each source document
#    separately, rather than the top N globally. This is useful for comparison
#    questions across documents (e.g., "How has Stone Ridge's philosophy changed
#    over time?"). When enabled, NUM_CONTEXT_CHUNKS specifies the number of chunks
#    to retrieve per document. Results are sorted using a two-step sort: first by
#    source name to group chunks from the same document together, then by score
#    descending within each group.
#


# =============================================================================
# CONSTANTS
# =============================================================================

# Maximum number of characters per chunk
CHUNK_SIZE = 1000

# Number of overlapping characters between consecutive chunks (helps preserve context at boundaries)
CHUNK_OVERLAP = 200

# Chunking strategy: "character" splits at fixed intervals, "paragraph" splits on newlines
SPLIT_MODE = "character"

# Number of most similar chunks to retrieve (per document if CONTEXT_PER_DOCUMENT is True)
NUM_CONTEXT_CHUNKS = 3

# When True, retrieve NUM_CONTEXT_CHUNKS from each source document separately
CONTEXT_PER_DOCUMENT = True

# Response style passed to the LLM (e.g., "detailed", "concise", "technical")
RESPONSE_STYLE = "detailed"

# Response length passed to the LLM (e.g., "brief", "efficient", "comprehensive")
RESPONSE_LENGTH = "efficient"

# Default question to ask the RAG application
#QUERY = "What is Stone Ridge's investment philosophy?"
QUERY = "Has Stone Ridge's investment philosophy evolved over the years?"


# =============================================================================
# IMPORTS
# =============================================================================

import os
import certifi
import asyncio
import nest_asyncio

from aimakerspace.text_utils import PDFFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase, cosine_similarity, pearson_correlation
from aimakerspace.ai_utils.prompts import UserRolePrompt, SystemRolePrompt
from aimakerspace.ai_utils.chatmodel import ChatAnthropic

nest_asyncio.apply()

# Distance metric for vector similarity: cosine_similarity or pearson_correlation
DISTANCE_METRIC = pearson_correlation


# =============================================================================
# PROMPT TEMPLATES
# =============================================================================

RAG_SYSTEM_TEMPLATE = """You are a helpful investor letter assistant that answers questions about Stone Ridge's investment philosophy, market insights, and strategic outlook based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in the investor letter"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a reminder that this is for informational purposes only and not investment advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""


# =============================================================================
# CLASSES
# =============================================================================

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatAnthropic, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores
        self.rag_system_prompt = SystemRolePrompt(RAG_SYSTEM_TEMPLATE)
        self.rag_user_prompt = UserRolePrompt(RAG_USER_TEMPLATE)

    def run_pipeline(self, user_query: str, k: int = 3, distance_metric=None, **system_kwargs) -> dict:
        if distance_metric is None:
            distance_metric = DISTANCE_METRIC
        
        if CONTEXT_PER_DOCUMENT:
            # Get unique sources and retrieve k chunks from each (sorted for chronological order)
            sources = sorted(self.vector_db_retriever.get_unique_metadata_values("source"))
            context_list = []
            for source in sources:
                results = self.vector_db_retriever.search_by_text(
                    user_query, k=k, distance_metric=distance_metric,
                    metadata_filter={"source": source}
                )
                context_list.extend(results)
            # Sort by source to group chunks together, then by score within each group
            context_list = sorted(context_list, key=lambda x: (x[2].get("source", ""), -x[1]))
        else:
            context_list = self.vector_db_retriever.search_by_text(
                user_query, k=k, distance_metric=distance_metric
            )
            context_list = sorted(context_list, key=lambda x: x[1], reverse=True)

        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score, metadata) in enumerate(context_list, 1):
            source = metadata.get("source", "Unknown")
            context_prompt += f"[Source: {source}]\n{context}\n\n---\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Log the context being sent to the LLM
        print("=" * 80)
        print("CONTEXT SENT TO LLM:")
        print("=" * 80)
        print(context_prompt)
        
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = self.rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = self.rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
        }


# =============================================================================
# FUNCTIONS
# =============================================================================

def zscaler_ssl_setup():
    """Configure SSL certificates to work with Zscaler corporate network."""
    zscaler_cert = "/Users/ari.packer/repos/sidekick/zscaler.pem"
    combined_cert = "/tmp/combined_certs.pem"

    with open(combined_cert, "w") as outfile:
        with open(certifi.where(), "r") as certifi_file:
            outfile.write(certifi_file.read())
        with open(zscaler_cert, "r") as zscaler_file:
            outfile.write(zscaler_file.read())

    os.environ['REQUESTS_CA_BUNDLE'] = combined_cert
    os.environ['SSL_CERT_FILE'] = combined_cert
    os.environ['CURL_CA_BUNDLE'] = combined_cert


def run_rag_application():
    """Build and run the RAG application for Stone Ridge investor letters."""
    documents = load_documents("data")
    print(f"Loaded {len(documents)} documents")

    split_documents, metadata_list = chunk_documents(documents)
    print(f"Split into {len(split_documents)} chunks")

    vector_db = init_vector_db(split_documents, metadata_list)
    print(f"Vector database built with {len(vector_db.vectors)} vectors")

    chat_llm = ChatAnthropic()

    rag_pipeline = RetrievalAugmentedQAPipeline(
        vector_db_retriever=vector_db,
        llm=chat_llm,
        response_style=RESPONSE_STYLE,
        include_scores=True
    )

    result = rag_pipeline.run_pipeline(
        QUERY,
        k=NUM_CONTEXT_CHUNKS,
        response_length=RESPONSE_LENGTH
    )

    print("=" * 80)
    print("RESPONSE:")
    print("=" * 80)
    print(result['response'])
    print("\n")
    print(f"Context Count: {result['context_count']}")
    print(f"Similarity Scores: {result['similarity_scores']}")


def load_documents(path: str) -> list:
    """Load PDF documents and filter out legal disclaimers."""
    pdf_loader = PDFFileLoader(path)
    documents = pdf_loader.load_documents()
    # Filter out Risk Disclosures while preserving metadata
    documents = [(text.split('Risk Disclosures')[0], source) for text, source in documents]
    return documents


def chunk_documents(documents: list) -> tuple:
    """Split documents into chunks using paragraph-based splitting."""
    text_splitter = CharacterTextSplitter(CHUNK_SIZE, CHUNK_OVERLAP, SPLIT_MODE)
    split_documents, metadata_list = text_splitter.split_texts_with_metadata(documents)
    return split_documents, metadata_list


def init_vector_db(split_documents: list, metadata_list: list) -> VectorDatabase:
    """Initialize vector database with document embeddings and metadata."""
    vector_db = VectorDatabase()
    vector_db = asyncio.run(vector_db.abuild_from_list(split_documents, metadata_list))
    return vector_db


# =============================================================================
# MAIN
# =============================================================================

zscaler_ssl_setup()
run_rag_application()